In [1]:
from typing import List, Sequence
from config import OPENAI_API_KEY, TAVILY_API_KEY, LANGCHAIN_TRACING_V2, LANGCHAIN_API_KEY
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph
from langchain_openai import ChatOpenAI

In [2]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
        "You are a viral twitter influencer grading a tweet. Generate critique and recommedations for the user's tweet."
        "Always provide detailed recommendation, including requests for length, virality, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
        
    ]
)

In [3]:
generation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
        "You are a twitter techie influencer assistant tasked with writing excellent twitter posts."
         "Generate the best twitter post possibile for the user's request."
         "If the user provides critique, respond with a revised version of your previous attempts."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [4]:
llm = ChatOpenAI()
generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm

In [5]:
REFLECT = "reflect"
GENERATE = "generate"

In [6]:
def generation_node (state: Sequence[BaseMessage]):
    print("\nGeneration Node:")
    print(state)
    print("*"*80)
    return generate_chain.invoke({"messages": state})

In [18]:
def reflection_node (state: Sequence[BaseMessage]) -> List[BaseMessage]:
    print("\nReflection Node:")
    print(state)
    print("*"*80)
    res = reflect_chain.invoke({"messages": state})
    return [HumanMessage(content=res.content)]

In [19]:
builder = MessageGraph()
builder.add_node(GENERATE, generation_node)
builder.add_node(REFLECT, reflection_node)
builder.set_entry_point(GENERATE)

In [20]:
def should_continue(state: List[BaseMessage]) -> str:
    if len(state) > 6:
        return END
    return REFLECT

In [21]:
builder.add_conditional_edges(GENERATE, should_continue)
builder.add_edge(REFLECT, GENERATE)
graph = builder.compile()

In [22]:
print(graph.get_graph())

DrawableGraph(nodes={'__start__': Node(id='__start__', data=<class 'pydantic.v1.main.LangGraphInput'>), '__end__': Node(id='__end__', data=<class 'pydantic.v1.main.LangGraphOutput'>), 'generate': Node(id='generate', data=generate(recurse=True)), 'reflect': Node(id='reflect', data=reflect(recurse=True))}, edges=[Edge(source='__start__', target='generate', data=None, conditional=False), Edge(source='reflect', target='generate', data=None, conditional=False), Edge(source='generate', target='reflect', data=None, conditional=True), Edge(source='generate', target='__end__', data=None, conditional=True)])


In [23]:
inputs = HumanMessage(content= """
Make this tweet better:
    @LangChainAI
- newly Tool Calling feature is seriously underrated.
After a long wait, It's here - making implementation of agents across different models with function calling - super easy.
Made a video coverign their newest blog post
""")

In [24]:
response = graph.invoke(inputs)


Generation Node:
[HumanMessage(content="\nMake this tweet better:\n    @LangChainAI\n- newly Tool Calling feature is seriously underrated.\nAfter a long wait, It's here - making implementation of agents across different models with function calling - super easy.\nMade a video coverign their newest blog post\n", id='984ce53f-e72a-4786-a485-31c5fa9081d7')]
********************************************************************************

Reflection Node:
[HumanMessage(content="\nMake this tweet better:\n    @LangChainAI\n- newly Tool Calling feature is seriously underrated.\nAfter a long wait, It's here - making implementation of agents across different models with function calling - super easy.\nMade a video coverign their newest blog post\n", id='984ce53f-e72a-4786-a485-31c5fa9081d7'), AIMessage(content="Excited to announce the launch of @LangChainAI's new Tool Calling feature! 🎉 Finally, implementing agents across various models with function calling is a breeze. Check out the video c

In [25]:
response[-1]

AIMessage(content="Exciting news! 🚀 @LangChainAI's new Tool Calling feature is a game-changer, simplifying agent implementation across different models. Check out their latest blog post video for all the details. #AI #tech #innovation #LangChainAI #blogpost #technews Watch now!", response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 596, 'total_tokens': 658}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_a450710239', 'finish_reason': 'stop', 'logprobs': None}, id='run-74869bf7-54a4-40a7-beaf-7637f89b1e44-0')